<a href="https://colab.research.google.com/github/dh610/ai-intensive2/blob/main/lab9/Text%20Generation%20Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ai-intensive2
!git pull
%cd lab9

Mounted at /content/drive
/content/drive/MyDrive/ai-intensive2
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 14.61 KiB | 26.00 KiB/s, done.
From https://github.com/dh610/ai-intensive2
   f54b27c..d373c74  main       -> origin/main
Updating f54b27c..d373c74
Fast-forward
 lab9/In-Context-Learning.ipynb | 3983 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++-
 1 file changed, 3982 insertions(+), 1 deletion(-)
/content/drive/MyDrive/ai-intensive2/lab9


In [ ]:
!pip install -q transformers datasets

# 텍스트 생성

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.set_grad_enabled(False)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device).eval()

## Greedy generation

가장 높은 확률의 토큰을 선택한다

In [ ]:
from pprint import pprint

input_text = "안녕하세요. "

input_ids = tokenizer(input_text, return_tensors="pt", add_special_tokens=False).to(device)

pprint(input_ids["input_ids"])
pprint(tokenizer.batch_decode(input_ids["input_ids"], skip_special_tokens=False))

logits = model(input_ids["input_ids"]).logits

pprint(logits.shape)

next_tokens = logits[:, -1].argmax(-1)

pprint(next_tokens)
pprint(tokenizer.batch_decode(next_tokens))

tensor([[25906,  8702,  7801, 25856,   739]])
['안녕하세요. ']
torch.Size([1, 5, 51200])
tensor([214])
['^^']


## Multinomial Sampling
각 확률을 가중치로 사용해서 확률적으로 추출한다

In [ ]:
for i in range(10):
  print(torch.multinomial(torch.tensor([0.1, 0.5, 0.4]), 1))

tensor([1])
tensor([1])
tensor([2])
tensor([1])
tensor([2])
tensor([2])
tensor([0])
tensor([1])
tensor([1])
tensor([1])


In [ ]:
for i in range(10):
  next_tokens = torch.multinomial(logits[:, -1].softmax(-1), 1)
  pprint(next_tokens)
  pprint(tokenizer.batch_decode(next_tokens))

tensor([[214]])
['^^']
tensor([[7735]])
['뽀']
tensor([[216]])
['ㅠㅠ']
tensor([[7522]])
['멕']
tensor([[7570]])
['뭘']
tensor([[214]])
['^^']
tensor([[9705]])
['..']
tensor([[605]])
['ᄏ']
tensor([[6910]])
['괜']
tensor([[214]])
['^^']


## Top-K sampling

확률이 가장 높은 K개를 골라서 그 중에서 샘플링한다

1. top-k token을 얻어온다
2. top-k token을 제외한 나머지 값들의 확률을 모두 -inf으로 만든다(극단적인 값으로 낮춤)
3. 이후 샘플링하면 끝

2번을 하는 방법
k번째 확률값을 가져온다


In [ ]:
A = torch.tensor([
          [0.5, 0.3, 0.4, 0.1],
          [0.9, 0.1, 0.01, 0.5]
          ])
k = 2
values, indices = A.topk(k)
k_value = values[:, -1].unsqueeze(-1)
print(k_value)
print(A < k_value)

tensor([[0.4000],
        [0.5000]])
tensor([[False,  True, False,  True],
        [False,  True,  True, False]])


In [ ]:
def filter_topk(logits, k=50):
  logits = logits.clone()
  lowest_value = - float('Inf')

  values, indices = logits.topk(k)

  # k 번째로 높은 값의 value를 가져온다
  k_value = values[:, -1].unsqueeze(-1) # [batch, 1]

  # k_value 보다 낮은 확률값은 전부 lowest_value 로 만들어버리면 끝!
  logits = logits.masked_fill(logits < k_value, lowest_value)
  return logits


for k in [1, 2, 3]:
  print(f"k = {k}", filter_topk (
      torch.tensor([
          [0.5, 0.3, 0.4, 0.1],
          [0.9, 0.1, 0.01, 0.5]
          ]),
      k = k
  ), sep="\n")

k = 1
tensor([[0.5000,   -inf,   -inf,   -inf],
        [0.9000,   -inf,   -inf,   -inf]])
k = 2
tensor([[0.5000,   -inf, 0.4000,   -inf],
        [0.9000,   -inf,   -inf, 0.5000]])
k = 3
tensor([[0.5000, 0.3000, 0.4000,   -inf],
        [0.9000, 0.1000,   -inf, 0.5000]])


ㄴ 위 함수 정답 결과는 아래와 같아야 합니다
```
k = 1
tensor([[0.5000,   -inf,   -inf,   -inf],
        [0.9000,   -inf,   -inf,   -inf]])
k = 2
tensor([[0.5000,   -inf, 0.4000,   -inf],
        [0.9000,   -inf,   -inf, 0.5000]])
k = 3
tensor([[0.5000, 0.3000, 0.4000,   -inf],
        [0.9000, 0.1000,   -inf, 0.5000]])
```

In [ ]:
for i in range(10):
  next_logits = filter_topk(logits[:, -1], 50)

  next_tokens = torch.multinomial(next_logits.softmax(-1), 1)
  pprint(next_tokens)
  pprint(tokenizer.batch_decode(next_tokens))

## Nucleus Sampling (Top-p sampling)
Top-K 샘플링의 단점은, 확률분포에 따라 너무 확률이 작은 토큰도 top-k에 포함될 위험도 있다는 점입니다. 이를 보안하기 위해 top-p sampling이 제안됬습니다. 이 방법은 토큰을 확률 순으로 정렬한 다음, 확률들을 큰 값부터 더해가다가 확률의 합이 threshold를 넘을 경우 중단하고 그 전의 높은 확률의 토큰들만을 샘플링에 사용합니다.


```
예를 들어 아래와 같은 확률이 있을 때
[0.05, 0.5, 0.3, 0.15]

정렬하면
[0.5, 0.3, 0.15, 0.05]

누적합
[0.5, 0.8, 0.95, 1.0]

threshold = 0.6일 경우 (0.6을 초과하는 토큰까지 포함)
[0.5, 0.3]

threshold = 0.8일 경우 (0.8과 정확히 일치해도 그 이후 토큰까지 포함함)
[0.5, 0.3, 0.15]

```

In [ ]:
# Hint 1. torch.sort & torch.cumsum
A = torch.tensor([
    [0.05, 0.5, 0.3, 0.15],
    [0.7, 0.09, 0.2, 0.01]
    ])

sorted_logits, sorted_indices = torch.sort(A, descending=True) # 내림차순 정렬
print("sorted_logits:", sorted_logits)
print("sorted_indices:", sorted_indices)

# dim=1 으로 누적
cum_logits = torch.cumsum(sorted_logits, dim=-1)
print("cum_logits:", cum_logits)

target = cum_logits >= 0.6
print(target)
#shift
target[:, 1:] = target[:, :-1].clone()
target[:, 0] = False
print(target)

# dim=0 으로 누적
cum_logits = torch.cumsum(sorted_logits, dim=0)
print("cum_logits:", cum_logits)


sorted_logits: tensor([[0.5000, 0.3000, 0.1500, 0.0500],
        [0.7000, 0.2000, 0.0900, 0.0100]])
sorted_indices: tensor([[1, 2, 3, 0],
        [0, 2, 1, 3]])
cum_logits: tensor([[0.5000, 0.8000, 0.9500, 1.0000],
        [0.7000, 0.9000, 0.9900, 1.0000]])
tensor([[False,  True,  True,  True],
        [ True,  True,  True,  True]])
tensor([[False, False,  True,  True],
        [False,  True,  True,  True]])
cum_logits: tensor([[0.5000, 0.3000, 0.1500, 0.0500],
        [1.2000, 0.5000, 0.2400, 0.0600]])


In [ ]:
# Hint 2. torch.scatter (src의 값을 index를 바꾸어서 저장한다)

target = torch.zeros((1, 4), dtype=torch.int32) # [[0, 0, 0, 0]]
print(target)

target.scatter_(
    dim=-1,
    index=torch.tensor([
        [3,1,2,0], # 새로 저장할 index
        ]),
    src=torch.tensor([[11,22,33,44]],dtype=torch.int32),
    )

print(target)

In [ ]:

def filter_topp(logits, threshold, verbose=False, return_softmax=True):
  logits = logits.clone()
  lowest_value = - float('Inf')
  batch_size = logits.shape[0]

  # 1. 정렬 후 cumsum 을 이용해서 누적합 계산
  sorted_logits, sorted_indices = ??
  cumulative_probs = ??

  # 2. threshold 보다 누적합이 큰 구간을 찾는다.
  # ex) threshold = 0.5
  # before: [0.5000, 0.7500, 0.9000, 1.0000]
  # after: [ False True True True]
  if verbose:
    print("cumulative_probs", cumulative_probs)

  sorted_indices_to_remove = ??

  if verbose:
    print("sorted_indices", sorted_indices)
    print("sorted_indices_to_remove", sorted_indices_to_remove)

  # 3. threshold를 초과한 토큰까지는 포함해야 하므로 sorted_indices_to_remove를 한 칸 옆으로 shift 한다
  # before: [ False True True True]
  # after: [ False False True True]
  sorted_indices_to_remove ??

  if verbose:
    print("sorted_indices_to_remove (shifted)", sorted_indices_to_remove)

  #
  # indices_to_remove 에는 cumsum이 threshold 를 넘는 토큰은 True 아닌 토큰은 False가 저장되어야 합니다.
  # 예시) 처음 확률값: [0.5, 0.15, 0.25, 0.1]
  # sorted_indices: [0, 2, 1, 3]
  # cumulative_probs: [0.5000, 0.7500, 0.9000, 1.0000]
  # sorted_indices_to_remove: [False, False,  True,  True]
  #
  # 앞에 두 값은 False고, 뒤의 두 갑은 True입니다.
  # 즉 sorted_index에서 앞의 0,2 는 top_p에 포함되고, 뒤의 1, 3은 top_p에 포함되지 않는다는 의미입니다.
  # 이제 scatter_를 이용해서 sorted_indices_to_remove 를 sorted_indices 맞춰 아래처럼 위치를 수정하면 됩니다.
  # indices_to_remove: [False, True, False, True]
  indices_to_remove = torch.zeros_like(logits, dtype=sorted_indices_to_remove.dtype)
  indices_to_remove.scatter_( ??? )

  if verbose:
    print("indices_to_remove (scattered)", indices_to_remove)

  if return_softmax:
    logits = logits.log()
  # True 인 index에만 lowest_value를 적용하고, 다시 총합을 1로 만듭니다.
  logits[indices_to_remove] = lowest_value

  return logits.softmax(-1) if return_softmax else logits


p = 0.70
print(f"p = {p}", filter_topp (
    torch.tensor([
        [0.5, 0.15, 0.25, 0.1],
        ]),
    p,
    True,
    False
), sep="\n")

다음과 같은 결과가 나와야 합니다
```
cumulative_probs tensor([[0.5000, 0.7500, 0.9000, 1.0000]])
sorted_indices tensor([[0, 2, 1, 3]])
sorted_indices_to_remove tensor([[False,  True,  True,  True]])
sorted_indices_to_remove (shifted) tensor([[False, False,  True,  True]])
indices_to_remove (scattered) tensor([[False,  True, False,  True]])
p = 0.7
tensor([[0.5000,   -inf, 0.2500,   -inf]])
```

In [ ]:
# test code!
for p in [0.5, 0.75, 0.9, 0.99]:
  print(f"p = {p}", filter_topp (
      torch.tensor([
          [0.5, 0.25, 0.15, 0.1],
          [0.7, 0.01, 0.01, 0.28]
          ]),
      p,
      False,
      False
  ), sep="\n")

실행결과

```
p = 0.5
tensor([[0.5000, 0.2500,   -inf,   -inf],
        [0.7000,   -inf,   -inf,   -inf]])
p = 0.75
tensor([[0.5000, 0.2500, 0.1500,   -inf],
        [0.7000,   -inf,   -inf, 0.2800]])
p = 0.9
tensor([[0.5000, 0.2500, 0.1500, 0.1000],
        [0.7000,   -inf,   -inf, 0.2800]])
p = 0.99
tensor([[0.5000, 0.2500, 0.1500, 0.1000],
        [0.7000, 0.0100, 0.0100, 0.2800]])
```

# 자기회귀 언어모델

In [ ]:
@torch.no_grad()
def generate(prefix,
             max_new_tokens,
             temperature=1,
             top_k=1,
             top_p=0,
             device='cpu'
             ):
    prefix_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)
    past_key_values = None

    for _ in range(max_new_tokens):
        outputs = model(
            prefix_ids,
            past_key_values=past_key_values,
            )
        logits = outputs.logits
        past_key_values = outputs.past_key_values

        logits = logits[:, -1] / temperature
        logits = filter_topk(logits, top_k)
        logits = filter_topp(logits.softmax(-1), top_p)

        next_tokens = torch.multinomial(logits, num_samples=1)

        prefix_ids = torch.cat((prefix_ids, next_tokens), dim=1)

    print(tokenizer.batch_decode(prefix_ids)[0])


generate('안녕하세요~', 16, top_p=0.95, top_k=50, temperature=1.5)

In [ ]:

def model_generate(prefix,
             max_new_tokens,
             temperature=1,
             top_k=1,
             top_p=0,
             device='cpu'
             ):
    prefix_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)
    outputs = model.generate(prefix_ids, do_sample=True)
    print(tokenizer.batch_decode(outputs)[0])

model_generate('안녕하세요~', 16)

# K/V Caching
디코딩 과정에서, transformer block의 이전 step Key, Value는 다시계산할 필요가 없습니다. 따라서 이 값들은 캐싱해둔 뒤 계산에서 제외합니다. 이 방법을 KV caching이라고 부릅니다.

![](https://miro.medium.com/v2/resize:fit:720/1*8xqD4AYTwn6mQXNw0uhDCg.gif)
before K/V caching

![](https://miro.medium.com/v2/resize:fit:720/1*uyuyOW1VBqmF5Gtv225XHQ.gif)

Reference: https://medium.com/@joaolages/kv-caching-explained-276520203249
